[https://www.kaggle.com/riteshkrjha/riiid-quick-tabnet-vs-xgboost](https://www.kaggle.com/riteshkrjha/riiid-quick-tabnet-vs-xgboost)

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, roc_curve, classification_report
import tensorflow as tf

from LorisNet import *

In [2]:
train = pd.read_csv('./data/riid/train.csv',
                   usecols=[1, 2, 3, 4, 5, 7, 8, 9],
                   dtype={'timestamp': 'int64',
                          'user_id': 'int32',
                          'content_id': 'int16',
                          'content_type_id': 'int8',
                          'task_container_id': 'int16',
                          'answered_correctly':'int8',
                          'prior_question_elapsed_time': 'float32',
                          'prior_question_had_explanation': 'boolean'}
                   )

In [3]:
# Remove lectures and additional processing
train = train[train.content_type_id == False]

train = train.sort_values(['timestamp'],
                          ascending=True)
train.drop(['timestamp', 'content_type_id'],
           axis=1,
           inplace=True)

In [4]:
# Read Questions and Lectures
questions = pd.read_csv('./data/riid/questions.csv')
lectures = pd.read_csv('./data/riid/lectures.csv')

In [5]:
# Merge train with Questions
train = pd.merge(train,
                 questions,
                 left_on='content_id',
                 right_on='question_id',
                 how='left')

In [6]:
#Indicator for first question in a batch
train['firstQindicator'] = np.where(train['prior_question_elapsed_time'].isnull(),
                                    1,
                                    0)
train['prior_question_elapsed_time'] = np.where(train['prior_question_elapsed_time'].isnull(),
                                                0,
                                                train['prior_question_elapsed_time'])

In [7]:
train.head()

,user_id,content_id,task_container_id,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,question_id,bundle_id,correct_answer,part,tags,firstQindicator
0,115,5692,1,1,0.0,<NA>,5692,5692,3,5,151,1
1,1805962620,5547,0,0,0.0,<NA>,5547,5547,0,5,8,1
2,2015251289,4024,0,1,0.0,<NA>,4024,4024,0,5,173,1
3,867941388,6659,0,1,0.0,<NA>,6659,6659,3,5,53,1
4,867946278,3977,0,1,0.0,<NA>,3977,3977,2,5,177,1


In [8]:
# Remove unused columns
del train['question_id']
del train['bundle_id']
del train['correct_answer']
del train['tags']

In [9]:
import gc
gc.collect()

0

In [10]:
train.prior_question_had_explanation = train.prior_question_had_explanation.astype(object)
train.prior_question_had_explanation = np.where(train.prior_question_had_explanation == 'True',
                                                1,
                                                0)

In [11]:
# Sample 5M records
train = train.sample(n=5000000)

In [12]:
# train test split
xtrain, xvalid, ytrain, yvalid = train_test_split(train.drop(['answered_correctly'], axis=1), 
                                                  train['answered_correctly'],
                                                  random_state=42, 
                                                  test_size=0.2, 
                                                  shuffle=True)

In [13]:
# Train LorisNet
model = tf.keras.Sequential()
model.add(tf.keras.layers.BatchNormalization())
model.add(LorisLayer(4,
                     5,
                     20,
                     multi_attentive_transformer_activity_regularizer=tf.keras.regularizers.L2(.0),
                     kernel_regularizer=tf.keras.regularizers.L2(.0),
                     number_parallel_groups=2,
                     MLP_with_skip_connections_depth=4
                    ))
model.add(tf.keras.layers.Dense(1, 'sigmoid'))

model.compile(loss=tf.keras.losses.binary_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['acc'])
train_tensor = tf.data.Dataset.from_tensor_slices((xtrain.values,
                                                   ytrain.values)).batch(10000)

import math
def step_decay(epoch):
    initial_lrate = 0.02
    drop = 0.5
    epochs_drop = 2.
    lrate = initial_lrate * math.pow(drop,  
                                     math.floor((1+epoch)/epochs_drop))
    return lrate
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(step_decay)
history = model.fit(train_tensor,
                    epochs=12,
                    callbacks=[lr_scheduler],
                    verbose=1)

Epoch 1/12
400/400 [==============================] - 549s 1s/step - loss: 0.6445 - acc: 0.6562 - lr: 0.0200
Epoch 2/12
400/400 [==============================] - 588s 1s/step - loss: 0.6430 - acc: 0.6570 - lr: 0.0100
Epoch 3/12
400/400 [==============================] - 568s 1s/step - loss: 0.6430 - acc: 0.6570 - lr: 0.0100
Epoch 4/12
400/400 [==============================] - 546s 1s/step - loss: 0.6430 - acc: 0.6570 - lr: 0.0050
Epoch 5/12
400/400 [==============================] - 596s 1s/step - loss: 0.6430 - acc: 0.6570 - lr: 0.0050
Epoch 6/12
400/400 [==============================] - 575s 1s/step - loss: 0.6430 - acc: 0.6570 - lr: 0.0025
Epoch 7/12
353/400 [=========================>....] - ETA: 1:06 - loss: 0.6430 - acc: 0.6571

KeyboardInterrupt: 

In [ ]:
model.predict(xvalid.values)

In [14]:
p = model.predict(xvalid.values).flatten().round().astype(int)
print('\t\t\tCLASSIFICATIION METRICS: LBBM\n')
print(classification_report(yvalid, p))
score = roc_auc_score(yvalid, p)
print('ROC value is: {}'.format(score))

31250/31250 [==============================] - 192s 6ms/step
			CLASSIFICATIION METRICS: LBBM



C:\Users\loris\.virtualenvs\LorisNet-x5Ovuv2r\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\loris\.virtualenvs\LorisNet-x5Ovuv2r\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\loris\.virtualenvs\LorisNet-x5Ovuv2r\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

              precision    recall  f1-score   support

           0       0.00      0.00      0.00    342945
           1       0.66      1.00      0.79    657055

    accuracy                           0.66   1000000
   macro avg       0.33      0.50      0.40   1000000
weighted avg       0.43      0.66      0.52   1000000

ROC value is: 0.5
